# Computational Theory Problems

In [1]:
# Numerical arrays and methods
import numpy as np
import struct
import math
from unittest import result

# Problem 1: Binary Words and Operations
## Overview
Following the Secure Hash Standard PDF, we implement seven bitwise functions that are fundamental to the SHA hashing algorithms.

## Symbols and Operations:

### & (Bitwise AND operation)
Returns 1 only when both of the corresponding bits are 1. If either bit has a 0, the result will be 0.

2-bit Example: 1100 & 1010 = 1000

3-bit Example: 1100 & 1010 & 1110 = 1000

### | (Bitwise OR operation)
Returns 1 only when at least the corresponding bits are 1. It will return 0 when both or all bits are 0.

2-bit Example: 1100 | 1010 = 1110

3-bit Example: 1100 | 1010 | 1110 = 1110

### ^ (Bitwise XOR operation - exclusive-OR)
Returns 1 when an odd number of corresponding bits are 1. It will return 0 when an even number of bits are 1.

2-bit Example: 1100 ^ 1010 = 0110

3-bit Example: 1100 ^ 1010 ^ 1110 = 0110

### ~ (Bitwise complement operation)
Inverts all bits - changes all the 0s to 1s and vice versa. 

2-bit Example: ~1100 = 0011

3-bit Example: ~1100 & ~1010 & ~1110 = 0001

### << (Left-Shift operation)
Ignores the left-most n bits and pads the result with n zeros on the right. It essentially multiplies the bits by 2^n.

2-bit Example: 1100 << 2 = 0000

3-bit Example: 0011 << 3 = 1000

### >> (Right-Shift operation)
Ignores the right-most n bits and pads the result with n zeros on the left. It essentially divides the bits by 2^n.

2-bit Example: 1100 >> 2 = 0011

3-bit Example: 1000 >> 3 = 0001

In [2]:
# Helper functions 
def rotr(x, n):
    """
    Rotate right (circular right shift) the 32-bit integer x by n bits.

    Parameters:
        x (int): The 32-bit integer to rotate.
        n (int): The number of bits/positions to rotate.

    Returns:
        int: The rotated integer as a 32-bit integer.
    """
    x = np.uint32(x)
    return np.uint32((x >> n) | (x << (32 - n)))


def rotl(x, n):
    """
    Rotate left (circular left shift) the 32-bit integer x by n bits.

    Parameters:
        x (int): The 32-bit integer to rotate.
        n (int): The number of bits/positions to rotate.

    Returns:
        int: The rotated integer as a 32-bit integer.
    """
    x = np.uint32(x)
    return np.uint32((x << n) | (x >> (32 - n)))


def shr(x, n):
    """
    A right shift operation only shifts with zeros, no rotating.

    Parameters:
        x (int): The 32-bit integer to shift.
        n (int): The number of bits/positions to shift.

    Returns:
        int: The shifted integer as a 32-bit integer.
    """
    x = np.uint32(x)
    return np.uint32(x >> n)

In [3]:
# Parity(x, y, z) function

def Parity(x, y, z):
    """
    This function calculates the parity of three 32-bit unsigned integers.

    It will return the result of the bitwise XOR operation on the three inputs.

    Parameters:
        x (int): First 32-bit unsigned integer.
        y (int): Second 32-bit unsigned integer.
        z (int): Third 32-bit unsigned integer.

    Returns:
        int: The parity result as a 32-bit unsigned integer.
    """
    x = np.uint32(x)
    y = np.uint32(y)
    z = np.uint32(z)
    
    return np.uint32(x ^ y ^ z)

In [4]:
# Ch(x, y, z) Function
def Ch(x, y, z):
   """
   This function calculates the choice function of three 32-bit unsigned integers.

   It will return the result of the bitwise operation: (x AND y) XOR (NOT x AND z).

   Parameters:
         x (int): First 32-bit unsigned integer.
         y (int): Second 32-bit unsigned integer.
         z (int): Third 32-bit unsigned integer.
   Returns:
         int: The choice result as a 32-bit unsigned integer.
   """
   x = np.uint32(x) 
   y = np.uint32(y)
   z = np.uint32(z)
   
   return np.uint32((x & y) ^ (~x & z))

In [5]:
# Maj(x, y, z) Function
def Maj(x, y, z):
    """
    This function calculates the majority function of three 32-bit unsigned integers.

    It will return the result of the bitwise operation: (x AND y) XOR (x AND z) XOR (y AND z).

    Parameters:
        x (int): First 32-bit unsigned integer.
        y (int): Second 32-bit unsigned integer.
        z (int): Third 32-bit unsigned integer.

    Returns:
        int: The majority result as a 32-bit unsigned integer.
    """
    x = np.uint32(x)
    y = np.uint32(y)
    z = np.uint32(z)
    
    return np.uint32((x & y) ^ (x & z) ^ (y & z))

In [6]:
# Sigma0(x) Function E0
def Sigma0(x):
    """
    This function calculates the Sigma0 (E0) function for a 32-bit unsigned integer.

    It will return the result of the bitwise operations: ROTR(x, 2) XOR ROTR(x, 13) XOR ROTR(x, 22).

    Parameters:
        x (int): A 32-bit unsigned integer.

    Returns:
        int: The Sigma0 result as a 32-bit unsigned integer.
    """
    # Ensure x is treated as a 32-bit unsigned integer
    x = np.uint32(x)
    
    return np.uint32(rotr(x, 2) ^ rotr(x, 13) ^ rotr(x, 22))


In [7]:
# Sigma1(x) Function E1
def Sigma1(x):
    """
    This function calculates the Sigma1 (E1) function for a 32-bit unsigned integer.

    It will return the result of the bitwise operations: ROTR(x, 6) XOR ROTR(x, 11) XOR ROTR(x, 25).

    Parameters:
        x (int): A 32-bit unsigned integer.

    Returns:
        int: The Sigma1 result as a 32-bit unsigned integer.
    """
    # Ensure x is treated as a 32-bit unsigned integer
    x = np.uint32(x)
    
    return np.uint32(rotr(x, 6) ^ rotr(x, 11) ^ rotr(x, 25))

In [8]:
# Sigma0(x) Function O0
def sigma0(x):
    """
    This function calculates the sigma0 (o0) function for a 32-bit unsigned integer.

    It will return the result of the bitwise operations: ROTR(x, 7) XOR ROTR(x, 18) XOR SHR(x, 3).

    Parameters:
        x (int): A 32-bit unsigned integer.

    Returns:
        int: The sigma0 result as a 32-bit unsigned integer.
    """
    # Ensure x is treated as a 32-bit unsigned integer
    x = np.uint32(x)
    
    return np.uint32(rotr(x, 7) ^ rotr(x, 18) ^ shr(x, 3)) 

In [9]:
# Sigma1(x) Function O1
def sigma1(x):
    """
    This function calculates the sigma1 (o1) function for a 32-bit unsigned integer.

    It will return the result of the bitwise operations: ROTR(x, 17) XOR ROTR(x, 19) XOR SHR(x, 10).

    Parameters:
        x (int): A 32-bit unsigned integer.

    Returns:
        int: The sigma1 result as a 32-bit unsigned integer.
    """
    # Ensure x is treated as a 32-bit unsigned integer
    x = np.uint32(x)
    
    return np.uint32(rotr(x, 17) ^ rotr(x, 19) ^ shr(x, 10))

# Testing the functions in Problem 1

In [10]:
# Test Parity Function with hexadecimal inputs
print("Testing Parity Function with hexadecimal inputs:")
result = Parity(0xF, 0xF, 0xF)
print(f"Expected: 0xf")
print(f"Result: {hex(result)}" + "\n")

# Test 2
print("Test 2")
result = Parity(0xFFFFFFFF, 0xFFFFFFFF, 0x00000000)
print(f"Expected: 0x0")
print(f"Result: {hex(result)}" + "\n")

# Test 3
print("Test 3")
result = Parity(0xAAAAAAAA, 0x55555555, 0x00000000)
print(f"Expected: 0xffffffff")
print(f"Result: {hex(result)}" + "\n")


Testing Parity Function with hexadecimal inputs:
Expected: 0xf
Result: 0xf

Test 2
Expected: 0x0
Result: 0x0

Test 3
Expected: 0xffffffff
Result: 0xffffffff



In [11]:
# Choice Function Test with hexadecimal inputs
print("\nTesting Choice Function with hexadecimal inputs:")
result = Ch(0xF, 0x0, 0xF)
print(f"Expected: 0x0")
print(f"Result: {hex(result)}" + "\n")

# Test 2
print("Test 2")
result = Ch(0xFFFFFFFF, 0x00000000, 0xFFFFFFFF)
print(f"Expected: 0x0")
print(f"Result: {hex(result)}" + "\n")

# Test 3
print("Test 3")
result = Ch(0xAAAAAAAA, 0x55555555, 0xFFFFFFFF)
print(f"Expected: 0x55555555")
print(f"Result: {hex(result)}" + "\n")


Testing Choice Function with hexadecimal inputs:
Expected: 0x0
Result: 0x0

Test 2
Expected: 0x0
Result: 0x0

Test 3
Expected: 0x55555555
Result: 0x55555555



In [12]:
# Majority Function Test with hexadecimal inputs
print("\nTesting Majority Function with hexadecimal inputs:")
result = Maj(0xF, 0xF, 0x0)
print(f"Expected: 0xf")
print(f"Result: {hex(result)}" + "\n")

# Test 2
print("Test 2")
result = Maj(0xFFFFFFFF, 0x00000000, 0xFFFFFFFF)
print(f"Expected: 0xffffffff")
print(f"Result: {hex(result)}" + "\n")

# Test 3
print("Test 3")
result = Maj(0xAAAAAAAA, 0x55555555, 0xFFFFFFFF)
print(f"Expected: 0xffffffff")
print(f"Result: {hex(result)}" + "\n")


Testing Majority Function with hexadecimal inputs:
Expected: 0xf
Result: 0xf

Test 2
Expected: 0xffffffff
Result: 0xffffffff

Test 3
Expected: 0xffffffff
Result: 0xffffffff



In [13]:
# All Sigma Functions Tests
# Sigma0 (E0) Function Test with hexadecimal input
print("\nTesting Sigma0 (E0) Function")
result = Sigma0(0xFFFFFFFF)
print(f"Result: {hex(result)}" + "\n")

# Sigma1 (E1) Function Test with hexadecimal input
print("Testing Sigma1 (E1) Function")
result = Sigma1(0xFFFFFFFF)
print(f"Result: {hex(result)}" + "\n")

# sigma0 (o0) Function Test with hexadecimal input
print("Testing sigma0 (o0) Function")
result = sigma0(0xFFFFFFFF)
print(f"Result: {hex(result)}" + "\n")

# sigma1 (o1) Function Test with hexadecimal input
print("Testing sigma1 (o1) Function")
result = sigma1(0xFFFFFFFF)
print(f"Result: {hex(result)}" + "\n")


Testing Sigma0 (E0) Function
Result: 0xffffffff

Testing Sigma1 (E1) Function
Result: 0xffffffff

Testing sigma0 (o0) Function
Result: 0x1fffffff

Testing sigma1 (o1) Function
Result: 0x3fffff



# Problem 2: Fractional Parts of Cube Roots
## Overview
From the Secure Hash Standard PDF on page 11, we will use numpy to calculate the constants used in the SHA-224 & SHA-256 algorithms.
These are the first 32 bits of the fractional parts of the cube roots of the first 64 prime numbers.

### Implementation Steps:
1. Create a `primes(n)` function that generates the first n prime numbers.
2. Create a function to calculate the cube root of the first 64 primes.
3. Extract the fractional part and convert to 32-bit integers.
4. Format and display results in hexadecimal.
5. Verify against the Secure Hash Standard.

### Approach
The function to generate the prime numbers uses an iterative approach. When given a number, checks each number up to that limit by testing if it has any divisions other than 1 and the number itself.

The Sieve of Eratosthenes is another approach to find prime numbers of a specific number. It will mark the multiples of each of the prime number starting from 2, leaving only prime numbers unmarked.

Since we are looking for the first 64 prime numbers I am using the iterative approach.

In [14]:
# Function to generate prime numbers
def primes(n):
    """
    Generate a `n` list of prime numbers.

    Parameters:
        n (int): The number of prime numbers to generate.

    Returns:
        list: A list of prime numbers up to `n`.
    """
    primeList = []

    num = 2

    while len(primeList) < n:
        isPrime = True
        for i in range(2, int(math.sqrt(num)) + 1):
            if (num % i) == 0:
                isPrime = False
                break
        if isPrime:
            primeList.append(num)
        num += 1
    return primeList

# Test usage
print("Generate the first 10 prime numbers:")
prime_numbers = primes(10)
print(prime_numbers)

Generate the first 10 prime numbers:
[2, 3, 5, 7, 11, 13, 17, 19, 23, 29]


In [15]:
# Function to Calculate the Cube Root of the prime numbers
def cube_root(primes):
    """
    This function calculates the cube root of the prime numbers

    Parameters:
        prime (list): A list of prime numbers.

    Returns:
        list: A list of cube roots of the given prime numbers.
    """
    cube_root_list = []
    for p in primes:
        cube_root = p ** (1/3)
        cube_root_list.append(cube_root)
    return cube_root_list

# Test usage
print("\nCube roots of the first 10 prime numbers:")
cube_roots_list = cube_root(prime_numbers)
print(cube_roots_list)


Cube roots of the first 10 prime numbers:
[1.2599210498948732, 1.4422495703074083, 1.7099759466766968, 1.912931182772389, 2.2239800905693152, 2.3513346877207573, 2.571281590658235, 2.668401648721945, 2.8438669798515654, 3.072316825685847]


In [16]:
# Function to extract the fractional part
def fractional_parts(cube_roots):
    """
    This function extracts the fractional part of the cube roots.

    Parameters:
        cube_roots (list): A list of cube roots.

    Returns:
        list: A list of fractional parts of the given cube roots.
    """
    fractional_list = []
    for root in cube_roots:
        fractional_part = root - int(root)
        fractional_list.append(fractional_part)
    return fractional_list

# Test usage
print("\nFractional parts of the cube roots:")
fractional_parts_list = fractional_parts(cube_roots_list)
print(fractional_parts_list)


Fractional parts of the cube roots:
[0.2599210498948732, 0.4422495703074083, 0.7099759466766968, 0.9129311827723889, 0.22398009056931523, 0.35133468772075727, 0.5712815906582351, 0.6684016487219449, 0.8438669798515654, 0.07231682568584707]


In [17]:
# Function to extract 32 bits from the fractional parts
def extract_32bits(fractional_parts):
    """
    Extract the first 32 bits of the fractional part.
    
    Parameters:
        fractional_part (list): A list of fractional values (each between 0 and 1).
    
    Returns:
        int: The 32-bit integer value.
    """
    bits_list = []
    for frac in fractional_parts:
        # Scale the fractional part to the range of a 32-bit integer
        scaled_value = int(frac * (2**32))
        bits_list.append(scaled_value)
    return bits_list

# Test usage
print("\nExtracted 32 bits from the fractional parts:")
extracted_bits_list = extract_32bits(fractional_parts_list)
print(extracted_bits_list)


Extracted 32 bits from the fractional parts:
[1116352408, 1899447441, 3049323471, 3921009573, 961987163, 1508970993, 2453635748, 2870763221, 3624381080, 310598401]


In [18]:
# Function to convert the extracted 32-bits to hexadecimal
def convert_to_hexadecimal(bits_value):
    """
    Convert the 32-bit integer to hexadecimal format.
    
    Parameters:
        bits_value (int): The 32-bit integer
    
    Returns:
        str: Hexadecimal representation
    """
    hex_list = []
    for value in bits_value:
        hex_value = hex(value)
        hex_list.append(hex_value)
    return hex_list

# Test usage
print("\nConverted 32 bits to hexadecimal:")
hexadecimal_list = convert_to_hexadecimal(extracted_bits_list)
print(hexadecimal_list)


Converted 32 bits to hexadecimal:
['0x428a2f98', '0x71374491', '0xb5c0fbcf', '0xe9b5dba5', '0x3956c25b', '0x59f111f1', '0x923f82a4', '0xab1c5ed5', '0xd807aa98', '0x12835b01']


In [ ]:
# Problem 2 results with the first 64 prime numbers
# Sequence of operations
first_64_primes = primes(64)
cube_roots_64 = cube_root(first_64_primes)
fractional_parts_64 = fractional_parts(cube_roots_64)
extracted_bits_64 = extract_32bits(fractional_parts_64)
hexadecimal_64 = convert_to_hexadecimal(extracted_bits_64)

print("SHA-224 & 256 K Constants first 64 prime numbers:")
# This will print 8 hexadecimal values per line
for i in range(0, len(hexadecimal_64), 8):
    print(hexadecimal_64[i:i+8])


Problem 2 results with the first 64 prime numbers:
SHA-224 & 256 K Constants first 64 prime numbers:
['0x428a2f98', '0x71374491', '0xb5c0fbcf', '0xe9b5dba5', '0x3956c25b', '0x59f111f1', '0x923f82a4', '0xab1c5ed5']
['0xd807aa98', '0x12835b01', '0x243185be', '0x550c7dc3', '0x72be5d74', '0x80deb1fe', '0x9bdc06a7', '0xc19bf174']
['0xe49b69c1', '0xefbe4786', '0xfc19dc6', '0x240ca1cc', '0x2de92c6f', '0x4a7484aa', '0x5cb0a9dc', '0x76f988da']
['0x983e5152', '0xa831c66d', '0xb00327c8', '0xbf597fc7', '0xc6e00bf3', '0xd5a79147', '0x6ca6351', '0x14292967']
['0x27b70a85', '0x2e1b2138', '0x4d2c6dfc', '0x53380d13', '0x650a7354', '0x766a0abb', '0x81c2c92e', '0x92722c85']
['0xa2bfe8a1', '0xa81a664b', '0xc24b8b70', '0xc76c51a3', '0xd192e819', '0xd6990624', '0xf40e3585', '0x106aa070']
['0x19a4c116', '0x1e376c08', '0x2748774c', '0x34b0bcb5', '0x391c0cb3', '0x4ed8aa4a', '0x5b9cca4f', '0x682e6ff3']
['0x748f82ee', '0x78a5636f', '0x84c87814', '0x8cc70208', '0x90befffa', '0xa4506ceb', '0xbef9a3f7', '0xc67178f

In [ ]:
# Comparing generated K constants with the official SHA-224 & 256 K constants
officialKConstants = [0x428a2f98, 0x71374491, 0xb5c0fbcf, 0xe9b5dba5, 0x3956c25b, 0x59f111f1, 0x923f82a4, 0xab1c5ed5, 0xd807aa98, 0x12835b01, 0x243185be, 0x550c7dc3,0x72be5d74, 0x80deb1fe, 0x9bdc06a7, 0xc19bf174, 0xe49b69c1, 0xefbe4786, 0x0fc19dc6, 0x240ca1cc, 0x2de92c6f, 0x4a7484aa, 0x5cb0a9dc, 0x76f988da, 0x983e5152, 0xa831c66d, 0xb00327c8, 0xbf597fc7, 0xc6e00bf3, 0xd5a79147, 0x06ca6351, 0x14292967, 0x27b70a85, 0x2e1b2138, 0x4d2c6dfc, 0x53380d13, 0x650a7354, 0x766a0abb, 0x81c2c92e, 0x92722c85,
0xa2bfe8a1, 0xa81a664b, 0xc24b8b70, 0xc76c51a3, 0xd192e819, 0xd6990624, 0xf40e3585, 0x106aa070, 0x19a4c116, 0x1e376c08, 0x2748774c, 0x34b0bcb5, 0x391c0cb3, 0x4ed8aa4a, 0x5b9cca4f, 0x682e6ff3, 0x748f82ee, 0x78a5636f, 0x84c87814, 0x8cc70208, 0x90befffa, 0xa4506ceb, 0xbef9a3f7, 0xc67178f2]

print(officialKConstants)
print(hexadecimal_64)

# Compare my generated K constants with the official SHA-224 & 256 K constants
print("\nComparing generated K constants with the official SHA-224 & 256 K constants:")
if hexadecimal_64 == officialKConstants:
    print("The generated K constants match the official SHA-224 & 256 K constants.")
else:
    print("The generated K constants do NOT match the official SHA-224 & 256 K constants.")
    

[1116352408, 1899447441, 3049323471, 3921009573, 961987163, 1508970993, 2453635748, 2870763221, 3624381080, 310598401, 607225278, 1426881987, 1925078388, 2162078206, 2614888103, 3248222580, 3835390401, 4022224774, 264347078, 604807628, 770255983, 1249150122, 1555081692, 1996064986, 2554220882, 2821834349, 2952996808, 3210313671, 3336571891, 3584528711, 113926993, 338241895, 666307205, 773529912, 1294757372, 1396182291, 1695183700, 1986661051, 2177026350, 2456956037, 2730485921, 2820302411, 3259730800, 3345764771, 3516065817, 3600352804, 4094571909, 275423344, 430227734, 506948616, 659060556, 883997877, 958139571, 1322822218, 1537002063, 1747873779, 1955562222, 2024104815, 2227730452, 2361852424, 2428436474, 2756734187, 3204031479, 3329325298]
['0x428a2f98', '0x71374491', '0xb5c0fbcf', '0xe9b5dba5', '0x3956c25b', '0x59f111f1', '0x923f82a4', '0xab1c5ed5', '0xd807aa98', '0x12835b01', '0x243185be', '0x550c7dc3', '0x72be5d74', '0x80deb1fe', '0x9bdc06a7', '0xc19bf174', '0xe49b69c1', '0xefbe4

# Problem 3: Padding

# Problem 4: Hashes

# Problem 5: Passwords


# End